In [1]:
import numpy as np
import h5py
from tqdm import tqdm

In [2]:
with h5py.File("singleStep_60.h5", "r") as fl:
    Ntraj = np.array(fl["NSamples"])
    state1Grid = np.array(fl["state1Grid"])
    state2Grid = np.array(fl["state2Grid"])
    state2GridUT = np.array(fl["state2GridUT"])
    dispList = np.array(fl["dispList"])
    rateList = np.array(fl["rateList"])
    jmpSelects = np.array(fl["JumpSelection"])
    specRates = np.array(fl["vu"])
    dxList = np.array(fl["jumpDisps"])
    jList = np.array(fl["jumpFinSites"])

In [3]:
# load the crystal
import pickle
with open("supercellBCC.pkl", "rb") as fl:
    superBCC = pickle.load(fl)
crys = superBCC.crys
N_units = superBCC.superlatt[0,0]

In [4]:
dxR = np.load("nnJumpLatVecs.npy")
RtoSiteInd = np.load("RtoSiteInd.npy")
SiteIndtoR = np.load("SiteIndtoR.npy")

In [5]:
# begin tests
for traj in tqdm(range(Ntraj), position=0, leave=True):
    state1 = state1Grid[traj]
    state2 = state2Grid[traj]
    state2UT = state2GridUT[traj]
    
    # First check that the vacancy is zero in both states
    assert state1[0,0,0] == state2[0,0,0] == 2
    
    # Next check that the correct displacement was stored for the state selected
    dx = dxList[jmpSelects[traj][0]]
    dxtoR = np.dot(np.linalg.inv(crys.lattice), dx).astype(int)
    assert np.array_equal(dxtoR, dxR[jmpSelects[traj][0]])
    Rnew = dxtoR % N_units
    siteIndNew = RtoSiteInd[Rnew[0], Rnew[1], Rnew[2]]
    assert siteIndNew == jList[jmpSelects[traj][0]]
    specB = state1[Rnew[0], Rnew[1], Rnew[2]]
    for spec in range(dispList.shape[1]):
        if spec == specB:
            assert np.allclose(dispList[traj, specB], -dx)
        elif spec == 2:
            assert np.allclose(dispList[traj, spec], dx)
        else:
            assert np.allclose(dispList[traj, spec], np.zeros(3))
            
    
    assert state2UT[0,0,0] == specB  #, "\n{} \n{} \n{} \n{}".format(specB, Rnew, dx, dxtoR)
    assert state2UT[Rnew[0], Rnew[1], Rnew[2]] == 2
    
    # Check that all the rest of the sites are the same
    for R0 in range(N_units):
        for R1 in range(N_units):
            for R2 in range(N_units):
                if R0 == 0 and R1 == 0 and R2 == 0:
                    continue
                if R0 == Rnew[0] and R1 == Rnew[1] and R2 == Rnew[2]:
                    continue
                assert state2UT[R0, R1, R2] == state1[R0, R1, R2]
    
    # Next check that state2 has been translated correctly
    dR = Rnew
    
    for R0 in range(N_units):
        for R1 in range(N_units):
            for R2 in range(N_units):
                R0T = (R0 + dR[0]) % N_units
                R1T = (R1 + dR[1]) % N_units
                R2T = (R2 + dR[2]) % N_units
                assert state2[R0, R1, R2] == state2UT[R0T, R1T, R2T], "\n{} {} {}".format(R0, R1, R2)
    
    # Next, check that the correct escape rate was computed
    rate = 0.
    for jmp in range(dxList.shape[0]):
        dx = dxList[jmp]
        dxtoR = np.dot(np.linalg.inv(crys.lattice), dx).astype(int)
        Rnew = dxtoR % N_units
        spec = state1[Rnew[0], Rnew[1], Rnew[2]]
        rate += specRates[spec]
    assert np.allclose(rate, rateList[traj])

100%|██████████| 100000/100000 [05:31<00:00, 302.00it/s]


In [6]:
state0 = state1Grid[0].reshape(512)

In [10]:
for stateInd in tqdm(range(state1Grid.shape[0]), position=0, leave=True):
    stateFlat = state1Grid[stateInd].reshape(512)
    for siteInd in range(N_units*N_units*N_units):
        R = SiteIndtoR[siteInd]
        assert state1Grid[stateInd, R[0], R[1], R[2]] == stateFlat[siteInd]
        
for stateInd in tqdm(range(state2Grid.shape[0]), position=0, leave=True):
    stateFlat = state2Grid[stateInd].reshape(512)
    for siteInd in range(N_units*N_units*N_units):
        R = SiteIndtoR[siteInd]
        assert state2Grid[stateInd, R[0], R[1], R[2]] == stateFlat[siteInd]

100%|██████████| 100000/100000 [00:56<00:00, 1759.28it/s]
